# Problem 14.13 Thermodynamic Cycle Calculation Using a High-Precision EOS

A thermodynamic cycle with water as the working fluid consists of the following steps:
    
* Constant-pressure heating to P1 = 100 bar and T1 = 350 °C
* Isentropic expansion of the gas in a turbine to P2 = 1 bar (reversible; efficiency = 100%)
* Constant pressure condensation
* Isentropic compression of the liquid to P4 = 100 bar

What is the thermal efficiency of the process?

$$ \eta_{th}  = -\frac{P_{12} + P_{34}}{Q_{41}}$$

## Solution

This is quite straightforward.

In [1]:
import numpy as np
from thermo import FlashPureVLS, IAPWS95Liquid, IAPWS95Gas, iapws_constants, iapws_correlations
from scipy.integrate import quad
import numpy as np

T1 = 350 + 273.15
P1 = 100*1e5
P2 = 1e5
# Entropy conserved in step 2 as well
VF3 = 0
P3 = P2

P4 = P1
# entropy conserved in step 5 as well


liquid = IAPWS95Liquid(T=T1, P=P1, zs=[1])
gas = IAPWS95Gas(T=T1, P=P1, zs=[1])
flasher = FlashPureVLS(iapws_constants, iapws_correlations, gas, [liquid], [])


stage_1 = flasher.flash(P=P1, T=T1)
stage_2 = flasher.flash(P=P2, S=stage_1.S())
stage_3 = flasher.flash(VF=VF3, P=P3)
stage_4 = flasher.flash(P=P4, S=stage_3.S())

expander_duty = stage_2.H() - stage_1.H()
pump_duty = stage_4.H() - stage_3.H()
heating_duty = stage_1.H() - stage_4.H()
cooling_duty = stage_3.H() - stage_2.H()
heating_duty, cooling_duty, expander_duty, pump_duty

(44969.97634439414,
 -31180.343551697508,
 -13975.281899345828,
 185.64910664919353)

In [2]:
# it is easy to check the cycle converged
cycle_error = sum([heating_duty, cooling_duty, expander_duty, pump_duty])
cycle_error

-9.094947017729282e-13

In [3]:
# Not quite sure what definition is being suggested by the textbook
eta_th = -expander_duty/heating_duty
print(f'The thermal efficiency is {eta_th*100:.2f} %')

The thermal efficiency is 31.08 %
